In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
df_main = pd.read_csv("nzz_art1.csv", delimiter = "|", header = None)

In [3]:
#remove columns without title, text or url
df_main = df_main.dropna(subset=[0])
df_main = df_main.dropna(subset=[5])
df_main = df_main.dropna(subset=[6])

In [4]:
#setting date

df_main[2] = pd.to_datetime(df_main[2])
df_main[7] = df_main[2].dt.year
df_main = df_main[df_main[7] != 2021]
df_main = df_main.drop([7], axis =1)

df_main[3] = df_main[3].apply(str)
df_main[3] = df_main[3].str.replace(".", ":")

In [5]:
#drop duplicates
df_main = df_main.drop_duplicates(keep = "last")

In [6]:
#categories
categories = df_main[6].str.split("/",expand=True)
df_main[7] = categories[3]

In [8]:
#reset index
df_main.reset_index(drop=True)

,0,1,2,3,4,5,6,7
0,Soll ich mich impfen lassen? Die wichtigsten ...,Auch in Deutschland und der Schweiz wurden die...,2020-12-16,10:07,"Stephanie Kusma, Stephanie Lahrtz",Zwei Impfstoffe gegen Sars-CoV-2 wurden in der...,https://www.nzz.ch/wissenschaft/coronavirus-fr...,wissenschaft
1,Was Sie zur SwissCovid-App wissen müssen,Um die Ausbreitung des Coronavirus einzudämmen...,2020-12-28,9:47,Jenni Thier,Die SwissCovid getaufte App ist seit dem 25. J...,https://www.nzz.ch/technologie/was-sie-zur-con...,technologie
2,Welche Gesichtsmaske ist die richtige?,"Im öffentlichen Verkehr, beim Einkaufen, in de...",2020-07-08,11:18,"Anja Lemcke (Grafik), Lena Bueche und Helga Ri...",Zu Beginn der Pandemie waren Masken noch ein k...,https://www.nzz.ch/wissenschaft/coronavirus-we...,wissenschaft
3,"Das Glossar zum Coronavirus –K-Wert, Positivi...",Die Liste der Begriffe rund ums Coronavirus wi...,2020-11-26,12:05,"Joana Kelén, Alexandra Kohler, Kaspar Manz",Inzwischen kennen wir viele Begriffe rund um d...,https://www.nzz.ch/visuals/coronavirus-diese-2...,visuals
4,"Wie lange es dauert, die Corona-Fallzahlen zu...",Derzeit stecken zehn Infizierte im Mittel neun...,2020-11-16,17:13,"Christian Speicher (Text), Joana Kelén (Grafik)",In ihrem jüngsten Lagebericht hat die Covid-19...,https://www.nzz.ch/wissenschaft/corona-halbwer...,wissenschaft
...,...,...,...,...,...,...,...,...
1587,Zwei Personen mit Verdacht auf Coronavirus im...,NaN,2020-01-26,17:59,NZZ-Redaktion,(tsm./sda) Erste Verdachtsfälle des Coronaviru...,https://www.nzz.ch/panorama/verdacht-auf-coron...,panorama
1588,Der Coronavirusausbruch in China ist laut der...,Bis anhin sind die meisten Erkrankungen und al...,2020-01-23,21:14,Stephanie Kusma,Die Weltgesundheitsorganisation (WHO) hat am D...,https://www.nzz.ch/wissenschaft/der-coronaviru...,wissenschaft
1589,Das Coronavirus drückt auf Chinas Dynamik,Wuhan ist ein bedeutendes Wirtschaftszentrum u...,2020-01-24,16:41,"Matthias Kamp, Michael Settelen",Der Wuhan Greenland Tower ist ein fast 500 Met...,https://www.nzz.ch/wirtschaft/das-corona-virus...,wirtschaft
1590,Warum das Coronavirus nicht nur in China die ...,Die Ausbreitung des Coronavirus bremst die chi...,2020-01-29,10:59,"Martin Kölling, Tokio",Die neue Viruserkrankung aus dem chinesischen ...,https://www.nzz.ch/wirtschaft/chinas-coronavir...,wirtschaft


In [9]:
#from categories to ressort topics
df_main[8] = ""
df_main[9] = df_main[7]
df_main[10] = ""

with open("ressorts_cat.bin", "rb") as data:
    ressorts = pickle.load(data)

df_main[9].replace(ressorts, inplace=True)
df_main[10].replace(ressorts, inplace=True)

In [10]:
#delete unworkables
with open("to_delete.bin", "rb")as data:
    to_delete = pickle.load(data)
    
df_main = df_main[~df_main[8].isin(to_delete)]
df_main = df_main[~df_main[7].isin(to_delete)]

In [12]:
#reorder columns to have consistent dataformat with the other article-outputs
df_main = df_main[[0,1,2,3,4,5,6,7, 8, 9, 10]]

In [14]:
df_main[12] = "nzz"
df_main[4] = df_main[4].str.replace("http\S+", "",regex = True)
df_main[4] = df_main[4].str.replace("Bild", "")

In [15]:
 df_main.to_csv("nzz_artikel.csv", header = None, sep = "|", index = False)